## Notebook 2: Network Analysis
This notebook builds user-user interaction graphs from subreddit comments

In [113]:
import pandas as pd
import praw
import numpy as np
import networkx as nx
from networkx.algorithms import bipartite
import math
import numpy as np
import igraph as ig
import leidenalg
import pickle
import matplotlib.pyplot as plt

reddit = praw.Reddit("project")
commentFrame = pd.read_pickle("comments.pkl")

### Constructing the user-user graph

In [114]:
graph = nx.Graph()
graph.add_nodes_from(commentFrame["original post"].unique(), bipartite = 0)
graph.add_nodes_from(commentFrame["author"].unique(), bipartite = 1)#average weighted upvotes, average sentiment


edges = zip(
    commentFrame["author"],
    commentFrame["original post"]
)

graph.add_edges_from((a, b) for a, b in edges)
print(graph.number_of_nodes())

uniqueUsers = [node for node, dic in graph.nodes(data = True) if dic["bipartite"] == 1]
userGraph = bipartite.weighted_projected_graph(graph, uniqueUsers)


3197


### Adding graph attributes

In [115]:
for u, v, data in userGraph.edges(data = True):
    raw_weight = data.get("weight", 1)

    deg_u = userGraph.degree(u, weight = "weight")
    deg_v = userGraph.degree(v, weight  = "weight")
    normalized = raw_weight / math.sqrt(deg_u * deg_v) if deg_u > 0 and deg_v > 0 else 0
user_sentiment = commentFrame.groupby("author")['sentimentLabel'].mean().to_dict()
nx.set_node_attributes(userGraph, user_sentiment, "sentiment")

### Saving graph for use in next notebook

In [116]:
components = nx.connected_components(userGraph)
giant_component = max(components, key = len)
G_Giant = userGraph.subgraph(giant_component)

with open("Giant_comp.pkl", "wb") as f:
    pickle.dump(G_Giant, f)